In [ ]:
different_types = !ls sample/ | cut -d '.' -f 2 | sort | uniq

In [ ]:
# show me files with 'qa' in postfix
!ls sample/*qa

In [ ]:
sample = !cat 'sample/qa/113333_second_order.json'

In [ ]:
from pprint import pprint
pprint(sample)

In [ ]:
!tree -d sample

In [49]:
import json

with open("sample/qa/113333_second_order.json", "r") as f:
    data = json.load(f)
print(data)

{'qa_pairs': [{'question_string': 'Is Blue curve the UnderDamped Case?', 'question_id': 15, 'color1_name': 'Blue', 'color2_name': '--None--', 'answer': 1, 'color1_id': 2, 'color1_rgb': [0, 0, 255], 'color2_id': -1, 'color2_rgb': [-1, -1, -1], 'image': '113333_second_order.png', 'annotations': '113333_second_order_annotations.json'}, {'question_string': 'Is Dark Salmon curve the OverDamped Case?', 'question_id': 16, 'color1_name': 'Dark Salmon', 'color2_name': '--None--', 'answer': 1, 'color1_id': 24, 'color1_rgb': [233, 150, 122], 'color2_id': -1, 'color2_rgb': [-1, -1, -1], 'image': '113333_second_order.png', 'annotations': '113333_second_order_annotations.json'}, {'question_string': 'Is Light Gold curve the UnderDamped Case?', 'question_id': 15, 'color1_name': 'Light Gold', 'color2_name': '--None--', 'answer': 0, 'color1_id': 36, 'color1_rgb': [255, 215, 0], 'color2_id': -1, 'color2_rgb': [-1, -1, -1], 'image': '113333_second_order.png', 'annotations': '113333_second_order_annotation

In [51]:
import json
import re

output = []
counter = 1

raw_data = data['qa_pairs']

for item in raw_data:
    data_string = json.dumps(item)
    item_dict = json.loads(data_string)
    new_dict = {}
    new_dict["answer"] = str(item["answer"])
    new_dict["image_filename"] = item["image"]
    new_dict["question"] = item["question_string"]
    new_dict["question_type"] = item["question_id"]
    pattern = r'(?<=_)(.*?)(?=\.png)'
    image_type = re.search(pattern, item["image"]).group(0)
    new_dict["image_type"] = image_type
    new_dict["question_index"] = item["color1_id"]
    new_dict["split"] = 'train'
    output.append(new_dict)
    counter += 1

print(output)

[{'answer': '1', 'image_filename': '113333_second_order.png', 'question': 'Is Blue curve the UnderDamped Case?', 'question_type': 15, 'image_type': 'second_order', 'question_index': 2, 'split': 'train'}, {'answer': '1', 'image_filename': '113333_second_order.png', 'question': 'Is Dark Salmon curve the OverDamped Case?', 'question_type': 16, 'image_type': 'second_order', 'question_index': 24, 'split': 'train'}, {'answer': '0', 'image_filename': '113333_second_order.png', 'question': 'Is Light Gold curve the UnderDamped Case?', 'question_type': 15, 'image_type': 'second_order', 'question_index': 36, 'split': 'train'}, {'answer': '0', 'image_filename': '113333_second_order.png', 'question': 'Is Light Gold curve the OverDamped Case?', 'question_type': 16, 'image_type': 'second_order', 'question_index': 36, 'split': 'train'}, {'answer': '1', 'image_filename': '113333_second_order.png', 'question': 'Is Blue the Second order system stable?', 'question_type': 21, 'image_type': 'second_order', 

In [ ]:
!find sample -maxdepth 1 -type f | wc -l

In [41]:
%%bash
find sample -type d | while read dir; do printf "%s : " "$dir"; find "$dir" -maxdepth 1 -type f | wc -l; done

sample : 20000
sample/qa : 20000


In [44]:
import os
os.system('find sample -type d | while read dir; do printf "%s : " "$dir"; find "$dir" -maxdepth 1 -type f | wc -l; done')

sample : 20000
sample/qa : 20000


0

In [48]:
import subprocess
output = subprocess.check_output('find sample -type d | while read dir; do printf "%s : " "$dir"; find "$dir" -maxdepth 1 -type f | wc -l; done', shell=True)
print(output.decode('utf-8'))

sample : 20000
sample/qa : 20000



NameError: name 'data' is not defined

In [63]:
%%writefile process_json.py

import json
import re
import pathlib
import logging
import argparse

# Set up logging configuration
logging.basicConfig(level=logging.INFO)

# Define constants for file extension and output file name
JSON_EXTENSION = '.json'
OUTPUT_FILE = 'output.json'

def process_json_file(file_path: pathlib.Path) -> list:
    """Process a JSON file and return a list of dictionaries with modified data.

    Args:
        file_path (pathlib.Path): The path to the JSON file.

    Returns:
        list: A list of dictionaries with modified data.
    """
    # Initialize an empty list to store the output data
    output_data = []

    # Open the JSON file and load the data
    with file_path.open('r') as f:
        data = json.load(f)
        raw_data = data['qa_pairs']

        # Loop over each item in raw_data and modify it
        for item in raw_data:
            # Create a new dictionary with modified data
            new_dict = {}
            new_dict["answer"] = str(item["answer"])
            new_dict["image_filename"] = item["image"]
            new_dict["question"] = item["question_string"]
            new_dict["question_type"] = item["question_id"]
            pattern = r'(?<=_)(.*?)(?=\.png)'
            image_type = re.search(pattern, item["image"]).group(0)
            new_dict["image_type"] = image_type
            new_dict["question_index"] = item["color1_id"]
            new_dict["split"] = 'train'

            # Append the new dictionary to the output list
            output_data.append(new_dict)

    # Return the output list
    return output_data

def process_json_files_in_directory(directory_path: pathlib.Path) -> None:
    """Process all the JSON files in a directory and save the output to a file.

    Args:
        directory_path (pathlib.Path): The path to the directory.
    """
    # Initialize an empty list to store all the output data
    all_output_data = []

    # Loop over all the files in the directory and filter by extension
    for file_path in directory_path.iterdir():
        if file_path.suffix == JSON_EXTENSION:
            # Process the JSON file and get the output data
            output_data = process_json_file(file_path)

            # Log a message with the file name and number of items processed
            logging.info(f"Processed {file_path.name} with {len(output_data)} items")

            # Extend the all_output_data list with the output data
            all_output_data.extend(output_data)

    # Save the all_output_data list to a JSON file
    with open(OUTPUT_FILE, 'w') as f:
        json.dump(all_output_data, f)

    # Log a message with the number of items saved
    logging.info(f"Saved {len(all_output_data)} items to {OUTPUT_FILE}")
    
if __name__ == '__main__':
    # Create an argument parser object
    parser = argparse.ArgumentParser(description="Process JSON files in a directory")

    # Add an argument for the directory path
    parser.add_argument('-d', '--directory', type=pathlib.Path, default='sample/qa', help="The path to the directory (default: sample/qa)")

    # Parse the arguments and get the directory path
    args = parser.parse_args()
    directory_path = args.directory

    # Process all the JSON files in the directory
    process_json_files_in_directory(directory_path)


Writing process_json.py


In [64]:
import ipywidgets as widgets
from IPython.display import display

directory_widget = widgets.Text(
    value='sample/qa',
    placeholder='Enter the directory path',
    description='Directory:',
    disabled=False
)

display(directory_widget)

Text(value='sample/qa', description='Directory:', placeholder='Enter the directory path')

In [68]:
import sys
sys.argv = ['process_json.py', '-d', directory_widget.value]

In [ ]:
if __name__ == '__main__' or '__file__' not in globals():
    # Create an argument parser object
    parser = argparse.ArgumentParser(description="Process JSON files in a directory")

    # Add an argument for the directory path
    parser.add_argument('-d', '--directory', type=pathlib.Path, default='sample/qa', help="The path to the directory (default: sample/qa)")

    # Parse the arguments and get the directory path
    args = parser.parse_args()
    directory_path = args.directory

    # Process all the JSON files in the directory
    process_json_files_in_directory(directory_path)


In [122]:
import json

with open('output.json', 'r') as f:
    data = json.load(f)

print(f"There are {len(data)} items in the output.json file")
import pandas as pd
data = pd.DataFrame.from_dict(data)
data.head()

There are 292992 items in the output.json file


,answer,image_filename,question,question_type,image_type,question_index,split
0,1,110469_second_order.png,Is Dark Gold curve the UnderDamped Case?,15,second_order,15,train
1,1,110469_second_order.png,Is Aqua curve the OverDamped Case?,16,second_order,0,train
2,0,110469_second_order.png,Is Aqua curve the UnderDamped Case?,15,second_order,0,train
3,0,110469_second_order.png,Is Dark Gold curve the OverDamped Case?,16,second_order,15,train
4,1,110469_second_order.png,Is Dark Gold the Second order system stable?,21,second_order,15,train


In [74]:
import json
import os

# Open the output.json file and load the data into a list
with open('output.json', 'r') as f:
    data = json.load(f)

# Loop over the data list and assign a number to each item
for i, item in enumerate(data):
    # Get the old image filename
    old_image_filename = item['image_filename']

    # Create a new image filename with the number and the extension
    new_image_filename = f'{i + 1000000}.png'

    # Update the item with the new image filename
    item['image_filename'] = new_image_filename

    # Get the old image file path
    old_image_file_path = os.path.join('sample', old_image_filename)

    # Create a new image file path with the new image filename
    new_image_file_path = os.path.join('sample', new_image_filename)

    # Rename the image file in the sample directory
    os.rename(old_image_file_path, new_image_file_path)

# Save the modified data list to a JSON file
with open('output.json', 'w') as f:
    json.dump(data, f)

FileNotFoundError: [Errno 2] No such file or directory: 'sample/110469_second_order.png' -> 'sample/1000001.png'

In [75]:
!mv 'sample/1000000.png' 'sample/110469_second_order.png'

3967.19s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [76]:
!cp -r sample/ image/

4282.99s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [77]:
import json
import os

# Open the output.json file and load the data into a list
with open('output.json', 'r') as f:
    data = json.load(f)

# Initialize an empty dictionary to store the mapping between old and new image filenames
image_filename_mapping = {}

# Loop over the data list and assign a number to each item
for i, item in enumerate(data):
    # Get the old image filename
    old_image_filename = item['image_filename']

    # Check if the old image filename is already in the mapping dictionary
    if old_image_filename in image_filename_mapping:
        # Get the new image filename from the mapping dictionary
        new_image_filename = image_filename_mapping[old_image_filename]
    else:
        # Create a new image filename with the number and the extension
        new_image_filename = f'{i + 100000}.png'

        # Update the mapping dictionary with the old and new image filenames
        image_filename_mapping[old_image_filename] = new_image_filename

        # Get the old image file path
        old_image_file_path = os.path.join('image', old_image_filename)

        # Create a new image file path with the new image filename
        new_image_file_path = os.path.join('image', new_image_filename)

        # Rename the image file in the sample directory
        os.rename(old_image_file_path, new_image_file_path)

    # Update the item with the new image filename
    item['image_filename'] = new_image_filename

# Save the modified data list to a JSON file
with open('output.json', 'w') as f:
    json.dump(data, f)


In [80]:
import json

with open('output.json', 'r') as f:
    data = json.load(f)

print(f"There are {len(data)} items in the output.json file")
import pandas as pd
data = pd.DataFrame.from_dict(data)
data.head(100)

There are 292992 items in the output.json file


,answer,image_filename,question,question_type,image_type,question_index,split
0,1,1000000.png,Is Dark Gold curve the UnderDamped Case?,15,second_order,15,train
1,1,1000000.png,Is Aqua curve the OverDamped Case?,16,second_order,0,train
2,0,1000000.png,Is Aqua curve the UnderDamped Case?,15,second_order,0,train
3,0,1000000.png,Is Dark Gold curve the OverDamped Case?,16,second_order,15,train
4,1,1000000.png,Is Dark Gold the Second order system stable?,21,second_order,15,train
...,...,...,...,...,...,...,...
95,0,1000084.png,Does Blue have the highest value of Steady state?,18,second_order,2,train
96,1,1000084.png,Does Coral have a max speed to reach output re...,19,second_order,9,train
97,1,1000084.png,Does Blue have a min speed to reach output res...,20,second_order,2,train
98,0,1000084.png,Does Maroon have a max speed to reach output r...,19,second_order,56,train


In [ ]:
import json
def reset_question_index(data):
    new_index = 1
    new_data = []
    for item in data:
        new_item = item.copy()
        new_item['question_index'] = new_index
        new_data.append(new_item)
        new_index += 1
    return new_data
with open('output.json', 'r') as f:
    data = json.load(f)
data = reset_question_index(data)
print(data[:100])
with open('output-q-idx.json', 'w') as f:
    json.dump(data, f)

In [83]:
import json

def reset_question_index(data):
    """Reset the question index for a list of dictionaries.

    Args:
        data (list): A list of dictionaries with question_index as a key.

    Returns:
        list: A new list of dictionaries with question_index starting from 1.
    """
    new_index = 1
    new_data = []

    for item in data:
        new_item = item.copy()
        new_item['question_index'] = new_index
        new_data.append(new_item)
        new_index += 1

    return new_data

with open('output.json', 'r') as f:
    data = json.load(f)

data = reset_question_index(data)

print(data[:100])

with open('output-q-idx.json', 'w') as f:
    json.dump(data, f)

[{'answer': '1', 'image_filename': '1000000.png', 'question': 'Is Dark Gold curve the UnderDamped Case?', 'question_type': 15, 'image_type': 'second_order', 'question_index': 1, 'split': 'train'}, {'answer': '1', 'image_filename': '1000000.png', 'question': 'Is Aqua curve the OverDamped Case?', 'question_type': 16, 'image_type': 'second_order', 'question_index': 2, 'split': 'train'}, {'answer': '0', 'image_filename': '1000000.png', 'question': 'Is Aqua curve the UnderDamped Case?', 'question_type': 15, 'image_type': 'second_order', 'question_index': 3, 'split': 'train'}, {'answer': '0', 'image_filename': '1000000.png', 'question': 'Is Dark Gold curve the OverDamped Case?', 'question_type': 16, 'image_type': 'second_order', 'question_index': 4, 'split': 'train'}, {'answer': '1', 'image_filename': '1000000.png', 'question': 'Is Dark Gold the Second order system stable?', 'question_type': 21, 'image_type': 'second_order', 'question_index': 5, 'split': 'train'}, {'answer': '1', 'image_file

In [84]:
import json

with open('output-q-idx.json', 'r') as f:
    data = json.load(f)

print(f"There are {len(data)} items in the output.json file")
import pandas as pd
data = pd.DataFrame.from_dict(data)
data.head(100)

There are 292992 items in the output.json file


,answer,image_filename,question,question_type,image_type,question_index,split
0,1,1000000.png,Is Dark Gold curve the UnderDamped Case?,15,second_order,1,train
1,1,1000000.png,Is Aqua curve the OverDamped Case?,16,second_order,2,train
2,0,1000000.png,Is Aqua curve the UnderDamped Case?,15,second_order,3,train
3,0,1000000.png,Is Dark Gold curve the OverDamped Case?,16,second_order,4,train
4,1,1000000.png,Is Dark Gold the Second order system stable?,21,second_order,5,train
...,...,...,...,...,...,...,...
95,0,1000084.png,Does Blue have the highest value of Steady state?,18,second_order,96,train
96,1,1000084.png,Does Coral have a max speed to reach output re...,19,second_order,97,train
97,1,1000084.png,Does Blue have a min speed to reach output res...,20,second_order,98,train
98,0,1000084.png,Does Maroon have a max speed to reach output r...,19,second_order,99,train


In [92]:
cd train

/notebooks/test/PREFIL/data/FigureQA/images/train


In [93]:
%%bash
max=0 # initialize max to zero
for file in * # loop over all files in current directory
do
  num=${file%.*} # remove the extension from file name
  if [[ $num =~ ^[0-9]+$ ]] # check if num is a valid number
  then
    if (( num > max )) # compare num with max
    then
      max=$num # update max if num is larger
    fi
  fi
done
echo "The largest number is $max" # print the result

The largest number is 99999


In [91]:
# biggest number in the directory
!ls | sort -n | tail -n 1

10794.93s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


val2


In [ ]:
!jq '.[-10:]' FigureQA_train_qa.json

In [104]:
ls

11869.00s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


LICENSE                 configs/            model.py*          run_cqa.py*
README.md*              cqa_dataloader.py*  output-q-idx.json  sample/
__pycache__/            data/               output.json        sample.zip
blockwise_densenet.py*  figureqa/           process_json.py
compute_metrics.py      image/              requirements.yml


In [105]:
# output-q-idx.json
import json

with open('output-q-idx.json', 'r') as f:
    data = json.load(f)

print(f"There are {len(data)} items in the output.json file")
import pandas as pd
data = pd.DataFrame.from_dict(data)
data.head(100)

There are 292992 items in the output.json file


,answer,image_filename,question,question_type,image_type,question_index,split
0,1,1000000.png,Is Dark Gold curve the UnderDamped Case?,15,second_order,1,train
1,1,1000000.png,Is Aqua curve the OverDamped Case?,16,second_order,2,train
2,0,1000000.png,Is Aqua curve the UnderDamped Case?,15,second_order,3,train
3,0,1000000.png,Is Dark Gold curve the OverDamped Case?,16,second_order,4,train
4,1,1000000.png,Is Dark Gold the Second order system stable?,21,second_order,5,train
...,...,...,...,...,...,...,...
95,0,1000084.png,Does Blue have the highest value of Steady state?,18,second_order,96,train
96,1,1000084.png,Does Coral have a max speed to reach output re...,19,second_order,97,train
97,1,1000084.png,Does Blue have a min speed to reach output res...,20,second_order,98,train
98,0,1000084.png,Does Maroon have a max speed to reach output r...,19,second_order,99,train


In [106]:
import json
import os

# Define the starting points
image_filename_start = 100000
question_index_start = 1327369

with open('output-q-idx.json', 'r') as f:
    data = json.load(f)

image_filename_mapping = {}
new_data = []

for i, item in enumerate(data):
    old_image_filename = item['image_filename']
    if old_image_filename in image_filename_mapping:
        new_image_filename = image_filename_mapping[old_image_filename]
    else:
        new_image_filename = f'{i + image_filename_start}.png'
        image_filename_mapping[old_image_filename] = new_image_filename
    old_image_file_path = os.path.join('image', old_image_filename)
    new_image_file_path = os.path.join('image', new_image_filename)
    os.rename(old_image_file_path, new_image_file_path)

    new_item = item.copy()
    new_item['image_filename'] = new_image_filename
    new_item['question_index'] = i + question_index_start
    new_data.append(new_item)

with open('ouput-second_order.json', 'w') as f:
    json.dump(new_data, f)

FileNotFoundError: [Errno 2] No such file or directory: 'image/1000000.png' -> 'image/100000.png'

In [111]:
mv 'image/100000.png' 'image/1000000.png'

15255.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [108]:
ls image/

14291.14s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


LICENSE                 configs/            model.py*          run_cqa.py*
README.md*              cqa_dataloader.py*  output-q-idx.json  sample/
__pycache__/            data/               output.json        sample.zip
blockwise_densenet.py*  figureqa/           process_json.py
compute_metrics.py      image/              requirements.yml


In [ ]:
import json
import os

# Define the starting points
image_filename_start = 100000
question_index_start = 1327369

with open('output-q-idx.json', 'r') as f:
    data = json.load(f)

image_filename_mapping = {}
new_data = []

for i, item in enumerate(data):
    old_image_filename = item['image_filename']
    if old_image_filename in image_filename_mapping:
        new_image_filename = image_filename_mapping[old_image_filename]
    else:
        new_image_filename = f'{i + image_filename_start}.png'
        image_filename_mapping[old_image_filename] = new_image_filename
    old_image_file_path = os.path.join('image', old_image_filename)
    new_image_file_path = os.path.join('image', new_image_filename)
    # Check if the file exists before renaming it
    if os.path.isfile(old_image_file_path):
        os.rename(old_image_file_path, new_image_file_path)

    new_item = item.copy()
    new_item['image_filename'] = new_image_filename
    new_item['question_index'] = i + question_index_start
    new_data.append(new_item)

# Use the same name as the input file for output
with open('ouput-second_order.json', 'w') as f:
    json.dump(new_data, f)

In [129]:
pwd

'/notebooks/test/PREFIL'

In [131]:
import json

with open('output.json', 'r') as f:
    data = json.loads(f.read())

print(f"There are {len(data)} items in the output.json file")
import pandas as pd
data = pd.DataFrame.from_dict(data)
data.head(100)

There are 292992 items in the output.json file


,answer,image_filename,question,question_type,image_type,question_index,split
0,1,110469_second_order.png,Is Dark Gold curve the UnderDamped Case?,15,second_order,15,train
1,1,110469_second_order.png,Is Aqua curve the OverDamped Case?,16,second_order,0,train
2,0,110469_second_order.png,Is Aqua curve the UnderDamped Case?,15,second_order,0,train
3,0,110469_second_order.png,Is Dark Gold curve the OverDamped Case?,16,second_order,15,train
4,1,110469_second_order.png,Is Dark Gold the Second order system stable?,21,second_order,15,train
...,...,...,...,...,...,...,...
95,0,106070_second_order.png,Does Blue have the highest value of Steady state?,18,second_order,2,train
96,1,106070_second_order.png,Does Coral have a max speed to reach output re...,19,second_order,9,train
97,1,106070_second_order.png,Does Blue have a min speed to reach output res...,20,second_order,2,train
98,0,106070_second_order.png,Does Maroon have a max speed to reach output r...,19,second_order,56,train


In [120]:
ls sample | head

16226.12s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


100000_second_order.png
100001_second_order.png
100002_second_order.png
100003_second_order.png
100004_second_order.png
100005_second_order.png
100006_second_order.png
100007_second_order.png
100008_second_order.png
100009_second_order.png
ls: write error


In [132]:
import json
import os

# Define the starting points
image_filename_start = 100000
question_index_start = 1327369

with open('output.json', 'r') as f:
    data = json.load(f)

image_filename_mapping = {}
new_data = []

# Use a separate variable for the new image filename index
new_image_filename_index = image_filename_start

for i, item in enumerate(data):
    old_image_filename = item['image_filename']
    if old_image_filename in image_filename_mapping:
        new_image_filename = image_filename_mapping[old_image_filename]
    else:
        # Use the new image filename index instead of i
        new_image_filename = f'{new_image_filename_index}.png'
        image_filename_mapping[old_image_filename] = new_image_filename
        # Increment the new image filename index only when a new old image filename is encountered
        new_image_filename_index += 1
    old_image_file_path = os.path.join('image', old_image_filename)
    new_image_file_path = os.path.join('image', new_image_filename)
    # Check if the file exists before renaming it
    if os.path.isfile(old_image_file_path):
        os.rename(old_image_file_path, new_image_file_path)

    new_item = item.copy()
    new_item['image_filename'] = new_image_filename
    new_item['question_index'] = i + question_index_start
    new_data.append(new_item)

with open('output.json', 'w') as f:
    json.dump(new_data, f)

In [133]:
with open('output.json', 'r') as f:
    data = json.loads(f.read())
    
print(f"There are {len(data)} items in the output.json file")

df = pd.DataFrame.from_dict(data)
df.head(100)

There are 292992 items in the output.json file


,answer,image_filename,question,question_type,image_type,question_index,split
0,1,100000.png,Is Dark Gold curve the UnderDamped Case?,15,second_order,1327369,train
1,1,100000.png,Is Aqua curve the OverDamped Case?,16,second_order,1327370,train
2,0,100000.png,Is Aqua curve the UnderDamped Case?,15,second_order,1327371,train
3,0,100000.png,Is Dark Gold curve the OverDamped Case?,16,second_order,1327372,train
4,1,100000.png,Is Dark Gold the Second order system stable?,21,second_order,1327373,train
...,...,...,...,...,...,...,...
95,0,100006.png,Does Blue have the highest value of Steady state?,18,second_order,1327464,train
96,1,100006.png,Does Coral have a max speed to reach output re...,19,second_order,1327465,train
97,1,100006.png,Does Blue have a min speed to reach output res...,20,second_order,1327466,train
98,0,100006.png,Does Maroon have a max speed to reach output r...,19,second_order,1327467,train


In [137]:
import json

with open('output.json', 'r') as f:
    data_1 = json.load(f)
    
with open('data/FigureQA/qa/FigureQA_train_qa.json', 'r') as f:
    data_2 = json.load(f)

data_2.extend(data_1)

with open('combined.json', 'w') as f:
    json.dump(data_2, f)

In [ ]:
import json
with open('combined.json', 'r') as f:
    data = json.loads(f.read())
    
data[-100:]

In [3]:
ls

LICENSE                 cqa_dataloader.py*       process_json.py
README.md*              data/                    requirements.yml
__pycache__/            figureqa/                run_cqa.py*
blockwise_densenet.py*  image/                   sample/
combined.json           model.py*                sample.zip
compute_metrics.py      ouput-second_order.json
configs/                output.json


In [ ]:
# data/FigureQA/images/train/

In [4]:
import json
import itertools
import random

# Load the combined json file
with open('combined.json', 'r') as f:
    data = json.load(f)

# Group the data by the image_filename key
groups = itertools.groupby(data, key=lambda x: x['image_filename'])

# Convert the groups into a list of lists
groups = [list(g) for k, g in groups]

# Shuffle the groups
random.shuffle(groups)

# Flatten the groups into a single list
shuffled_data = [item for group in groups for item in group]

# Reset the question_index for each item
for i, item in enumerate(shuffled_data):
    item['question_index'] = i + 1

# Save the shuffled data as a new json file
with open('shuffled.json', 'w') as f:
    json.dump(shuffled_data, f)


In [ ]:
!jq '.[-30:]' shuffled.json

In [ ]:
import json
import itertools
import random
import os

# Load the combined json file
with open('combined.json', 'r') as f:
    data = json.load(f)

# Group the data by the image_filename key
groups = itertools.groupby(data, key=lambda x: x['image_filename'])

# Convert the groups into a list of lists
groups = [list(g) for k, g in groups]

# Shuffle the groups
random.shuffle(groups)

# Flatten the groups into a single list
shuffled_data = [item for group in groups for item in group]

# Define the starting point for the new image filename index
new_image_filename_start = 0

# Use a dictionary to map the old image filenames to the new ones
image_filename_mapping = {}

# Use a list to store the updated data
updated_data = []

# Use a variable for the new image filename index
new_image_filename_index = new_image_filename_start

# Loop through the shuffled data
for i, item in enumerate(shuffled_data):
    # Get the old image filename
    old_image_filename = item['image_filename']
    # Check if it is already mapped to a new one
    if old_image_filename in image_filename_mapping:
        # Get the new image filename from the mapping
        new_image_filename = image_filename_mapping[old_image_filename]
    else:
        # Generate a new image filename based on the index
        new_image_filename = f'{new_image_filename_index}.png'
        # Store the mapping for future reference
        image_filename_mapping[old_image_filename] = new_image_filename
        # Increment the new image filename index only when a new old image filename is encountered
        new_image_filename_index += 1
    
    # Rename the image file if it exists
    old_image_file_path = os.path.join('image', old_image_filename)
    new_image_file_path = os.path.join('image', new_image_filename)
    if os.path.isfile(old_image_file_path):
        os.rename(old_image_file_path, new_image_file_path)

    # Update the item with the new image filename and question index
    item['image_filename'] = new_image_filename
    item['question_index'] = i + 1

    # Append the updated item to the list
    updated_data.append(item)

# Save the updated data as a new json file
with open('updated.json', 'w') as f:
    json.dump(updated_data, f)

In [8]:
pwd

'/notebooks/test/PREFIL'

In [12]:
import json
import itertools
import random
import os

# Load the shuffled json file
with open('shuffled.json', 'r') as f:
    data = json.load(f)

# Define the starting point for the new image filename index
new_image_filename_start = 0

# Use a dictionary to map the old image filenames to the new ones
image_filename_mapping = {}

# Use a list to store the updated data
updated_data = []

# Use a variable for the new image filename index
new_image_filename_index = new_image_filename_start

# Loop through the data
for i, item in enumerate(data):
    # Get the old image filename
    old_image_filename = item['image_filename']
    # Check if it is already mapped to a new one
    if old_image_filename in image_filename_mapping:
        # Get the new image filename from the mapping
        new_image_filename = image_filename_mapping[old_image_filename]
    else:
        # Generate a new image filename based on the index
        new_image_filename = f'{new_image_filename_index}.png'
        # Store the mapping for future reference
        image_filename_mapping[old_image_filename] = new_image_filename
        # Increment the new image filename index only when a new old image filename is encountered
        new_image_filename_index += 1
    
    # Rename the image file if it exists
    old_image_file_path = os.path.join('data/FigureQA/images/train/', old_image_filename)
    new_image_file_path = os.path.join('data/FigureQA/images/train/', new_image_filename)
    if os.path.isfile(old_image_file_path):
        os.rename(old_image_file_path, new_image_file_path)

    # Update the item with the new image filename and question index
    item['image_filename'] = new_image_filename
    item['question_index'] = i + 1

    # Append the updated item to the list
    updated_data.append(item)

# Save the updated data as a new json file
with open('updated.json', 'w') as f:
    json.dump(updated_data, f)

In [ ]:
# find data/FigureQA/images/train/ -name "*" -print0 | xargs -0 cp -t data/FigureQA/images/train_bak/
# 
# find data/FigureQA/images/train/ -name "*" -print0 | xargs -0 -P 4 cp -t data/FigureQA/images/train_bak/
# 

In [13]:
# !jq '.[:25]' updated.json

In [23]:
ls data/FigureQA/images/train/ | grep -e "16920.png"

116920.png


In [2]:
pwd

'/notebooks'

In [11]:
import json
import itertools
import random
import os
import shutil

# Load the combined json file
with open('shuffled.json', 'r') as f:
    data = json.load(f)

# Group the data by the image_filename key
groups = itertools.groupby(data, key=lambda x: x['image_filename'])

# Convert the groups into a list of lists
groups = [list(g) for k, g in groups]

# Shuffle the groups
random.shuffle(groups)

# Flatten the groups into a single list
shuffled_data = [item for group in groups for item in group]

# Define the starting point for the new image filename index
new_image_filename_start = 0

# Use a dictionary to map the old image filenames to the new ones
image_filename_mapping = {}

# Use a list to store the updated data
updated_data = []

# Use a variable for the new image filename index
new_image_filename_index = new_image_filename_start

# Create a new folder for the renamed image files
new_image_folder = 'data/FigureQA/images/train_new/'
os.makedirs(new_image_folder)

# Loop through the shuffled data
for i, item in enumerate(shuffled_data):
    # Get the old image filename
    old_image_filename = item['image_filename']
    # Check if it is already mapped to a new one
    if old_image_filename in image_filename_mapping:
        # Get the new image filename from the mapping
        new_image_filename = image_filename_mapping[old_image_filename]
    else:
        # Generate a new image filename based on the index
        new_image_filename = f'{new_image_filename_index}.png'
        # Store the mapping for future reference
        image_filename_mapping[old_image_filename] = new_image_filename
        # Increment the new image filename index only when a new old image filename is encountered
        new_image_filename_index += 1
    
    # Copy the image file to the new folder with the new name if it exists
    old_image_file_path = os.path.join('data/FigureQA/images/train/', old_image_filename)
    new_image_file_path = os.path.join(new_image_folder, new_image_filename)
    if os.path.isfile(old_image_file_path):
        shutil.copy(old_image_file_path, new_image_file_path)

    # Update the item with the new image filename and question index
    item['image_filename'] = new_image_filename
    item['question_index'] = i + 1

    # Append the updated item to the list
    updated_data.append(item)

# Save the updated data as a new json file
with open('updated.json', 'w') as f:
    json.dump(updated_data, f)

# Delete the old folder with the old image files
shutil.rmtree('data/FigureQA/images/train/')